# Data Analysis

Dans cette partie, nous allons traiter/analyser nos données et justifier nos choix pratiques/éthiques au niveau des features choisis.

In [12]:
import pandas as pd

#ouverture Mise en Dataframe des données
employee_survey_data_path = "datas/employee_survey_data.csv"
manager_survey_data_path = "datas/manager_survey_data.csv"
general_data_path = "datas/general_data.csv"
in_time_data_path = "datas/in_time.csv"
out_time_data_path = "datas/out_time.csv"

employee_survey_data = pd.read_csv(employee_survey_data_path)
manager_survey_data = pd.read_csv(manager_survey_data_path)
general_data = pd.read_csv(general_data_path)
in_time_data = pd.read_csv(in_time_data_path)
out_time_data = pd.read_csv(out_time_data_path)

Dans un premier temps, on regarde les données pour voir s'il y a des features manquantes pour certaines instances.

In [13]:
print("EMPLOYEE SURVEY DATA")
print(employee_survey_data.info())
print("---------------------------------------------------------")
print("MANAGER SURVEY DATA")
print(manager_survey_data.info())
print("---------------------------------------------------------")
print("GENERAL DATA")
print(general_data.info())
print("---------------------------------------------------------")
print("IN DATA")
print(in_time_data.info(max_cols=300))
print("---------------------------------------------------------")
print("OUT DATA")
print(out_time_data.info(max_cols=300))
print("---------------------------------------------------------")

EMPLOYEE SURVEY DATA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB
None
---------------------------------------------------------
MANAGER SURVEY DATA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB
None
---------------------------------